In [1]:
import datetime  # For datetime objects
import os.path  # To manage paths
from datetime import date, datetime, timedelta

# Import the backtrader platform
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats


In [2]:
class BuyAndHold_1(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.data)
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [3]:
cerebro = bt.Cerebro()

# Set our desired cash start
cerebro.broker.setcash(1000000.0)

idx = cerebro.addstrategy(BuyAndHold_1)
cerebro.addsizer(bt.sizers.AllInSizer)


In [4]:
# Create a Data Feed
PATH = "../data/investpy/data/index/"
data = bt.feeds.GenericCSVData(
    dataname=PATH+'S&P_500.csv',

    fromdate=datetime(2000, 1, 1),
    todate=datetime(2020, 12, 31),

    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),

    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
    openinterest=-1
)

In [5]:
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

start_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % start_portfolio_value)

results = cerebro.run(maxcpu=4)
strat = results[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

end_portfolio_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % end_portfolio_value)
pnl = end_portfolio_value - start_portfolio_value
print('PnL: %.2f' % pnl)

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = strat.analyzers.returns.get_analysis()['rnorm100']
sharpe = strat.analyzers.sharperatio.get_analysis()['sharperatio']
print(f"Max Drawdown: {maxdrawdown:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")


Starting Portfolio Value: 1000000.00
ROI:        193.25%
Final Portfolio Value: 2932465.90
PnL: 1932465.90
Max Drawdown: 56.65%
CAGR: 7.45%
Sharpe: 0.489


cerebro.plot()

#### Increment the portfolio with an amount of cash every month

In [6]:
class BuyAndHold_More(bt.Strategy):
    params = dict(
        monthly_cash=10000.0,  # amount of cash to buy every month
    )

    def start(self):
        self.cash_start = self.broker.get_cash()
        self.val_start = 10000.0

        # Add a timer which will be called on the 1st trading day of the month
        self.add_timer(
            bt.timer.SESSION_END,  # when it will be called
            monthdays=[1],  # called on the 1st day of the month
            monthcarry=True,  # called on the 2nd day if the 1st is holiday
        )

    def notify_timer(self, timer, when, *args, **kwargs):
        # Add the influx of monthly cash to the broker
        self.broker.add_cash(self.p.monthly_cash)

        # buy available cash
        target_value = self.broker.get_value() + self.p.monthly_cash
        self.order_target_value(target=target_value)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.cash_start) - 1.0
        print('ROI:        {:.2f}%'.format(self.roi))

In [7]:
cerebro = bt.Cerebro()

idx = cerebro.addstrategy(BuyAndHold_More)
cerebro.addsizer(bt.sizers.AllInSizer)

In [8]:
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

start_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % start_portfolio_value)

results = cerebro.run(maxcpu=4)
strat = results[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

end_portfolio_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % end_portfolio_value)
pnl = end_portfolio_value - start_portfolio_value
print('PnL: %.2f' % pnl)

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
print(f"Max Drawdown: {maxdrawdown:.2f}")


Starting Portfolio Value: 10000.00
ROI:        420.49%
Final Portfolio Value: 4214931.70
PnL: 4204931.70
Max Drawdown: 36.31
